## Filter2d Testbench

This notebook is to test the implementation of a 2D filter running on the programmable logic. The filter is used to make pictures special, like sharpening, bluring or edge detecting. The output is compared with a Python calculation of the openCV. It takes in img_in(grayscale, size=(128,128)), convolution kernel(3×3), rows and cols and gives out img_out(grayscale, size=(126,126)) which is calculated by the convolution kernel.

### Import third party libraries

In [ ]:
import cv2
import numpy as np
import time
from matplotlib import pyplot as plt

### Load image and show its size

In [ ]:
img = cv2.imread("./aojiao_128_128.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
print("灰度图尺寸：{}".format(gray.shape))
plt.imshow(gray,cmap='gray')
# Loading source image
src_image = gray

### Load IP

In [ ]:
from pynq import Overlay, allocate

overlay = Overlay("./filter2D.bit")

myfilter2d = overlay.filter2d_accel_0

### Define function to compare the effect between CV2 and customized IP

In [ ]:
def CV2vsIP(src_img, kernel):
    #process the same image with opencv and my harware IP
    fig_sharpen = plt.figure()
   
    # fig1: source image
    fig1 = fig_sharpen.add_subplot(131)
    fig1.title.set_text('original image')
    plt.imshow(src_image,cmap='gray')
    
    # fig2: opencv
    start_time = time.time()
    cv2_image = cv2.filter2D(src_image, -1, kernel)
    end_time = time.time()
    
    fig2 = fig_sharpen.add_subplot(132)
    fig2.title.set_text('CV2 filter2d image')
    plt.imshow(cv2_image,cmap='gray')

    print("OpenCV耗时：{}s".format(end_time - start_time))
    
    #fig3: define by IP
    # allocate memory
    rows, cols= src_image.shape
    input_buffer = allocate(shape=(rows*cols,), dtype='int')
    output_buffer = allocate(shape=(126*126,), dtype='int')
    input_kernel = allocate(shape=(9,), dtype='int')

    # input
    src_image_new = np.int32(src_image)
    kernel_new = np.int32(kernel)
    np.copyto(input_buffer, src_image_new.flatten())
    np.copyto(input_kernel, kernel_new.flatten())
    
    #control the fpga
    myfilter2d.s_axi_CTRL.write(0x10, rows)
    myfilter2d.s_axi_CTRL.write(0x18, cols)
    myfilter2d.s_axi_control.write(0x10,input_buffer.physical_address)
    myfilter2d.s_axi_control.write(0x1c,input_kernel.physical_address)
    myfilter2d.s_axi_control.write(0x28,output_buffer.physical_address)
    
    #start
    myfilter2d.s_axi_CTRL.write(0x00, 0x01)
    start_time = time.time()
    while True:
        reg = myfilter2d.s_axi_CTRL.read(0x00)
        if reg != 1:
            break
    end_time = time.time()

    print("FPGA耗时：{}s".format(end_time - start_time))
    
    output_img = output_buffer
    output_img = np.int32(output_img)
    fig3 = fig_sharpen.add_subplot(133)
    fig3.title.set_text('FPGA filter2d image')
    plt.imshow(output_img.reshape(126,126),cmap='gray')
    
    return

### Sharpening an image

In [ ]:
kernel = np.array([
  [0, -1, 0],
  [-1, 5, -1],
  [0, -1, 0]
])
CV2vsIP(src_image, kernel)

### Edge detection

In [ ]:
kernel = np.array([
  [-1, -1, -1],
  [-1, 9, -1],
  [-1, -1, -1]
])
CV2vsIP(src_image, kernel)

### Emboss filter

In [ ]:
kernel = np.array([
  [-1, -1, 0],
  [-1, 0, 1],
  [0, 1, 1]
])
CV2vsIP(src_image, kernel)